In [ ]:
#| default_exp embed_images

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

In [ ]:
#| export

from pathlib import Path
from PIL import Image
import numpy as np
import torch
from tqdm import tqdm
from loguru import logger
from transformers import pipeline
import daft

from bedmap.config import Cfg
from bedmap.step import step

In [ ]:
#| export

BATCH_SIZE = 4

In [ ]:
#| export

def images_from_paths(pathlist):
    return (Image.open(p.as_posix()).convert("RGB").copy() for p in pathlist)

In [ ]:
#| export

def embed_images(imagepaths : list[Path],
                 model_name : str = "timm/vit_small_patch14_reg4_dinov2.lvd142m",
                 batch_size : int = 4
                 ) -> list[np.array]:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pipe = pipeline(task="image-feature-extraction",
                    model=model_name, device=device, pool=True, use_fast=True)

    # logger.info("Starting embedding pipeline.")
    embeddings = []

    for out in tqdm(pipe(images_from_paths(imagepaths), batch_size=batch_size),
                    total=len(imagepaths)//batch_size):
        embeddings += out

    # logger.info("Done with embedding pipeline.")

    return np.array(embeddings)

In [ ]:
# df = daft.from_pydict({"img_path": ["file://cae.", "file:////acea/"]})
# df = df.with_column("img_path_trim",
#                     df["img_path"].str.replace(pattern="^file:/+", replacement="/", regex=True)
#                    )
# df.select(daft.col("img_path_trim")).to_pylist()



In [ ]:
#| export

@step(requires=["img_path"], provides=["embeddings"])
def create_embeddings_col(df: daft.DataFrame, model_name: str, batch_size: int) -> daft.DataFrame:
    """
    Embed images for a given dataframe.
    """
    ## daft encodes paths as URIs, always starting with file://
    df = df.with_column("img_path_nouri", df["img_path"].str.replace(
        pattern="^file://", replacement="", regex=True))
    paths = [Path(r["img_path_nouri"]) for r in df.select("img_path_nouri").to_pylist()]
    embeds = embed_images(paths, model_name=model_name, batch_size=batch_size)
    # fixed_size_list lets us use normal arrow methods to calculate length later
    embeds_type = daft.DataType.fixed_size_list(daft.DataType.float32(), embeds.shape[-1])
    embeds_series = daft.Series.from_numpy(embeds).cast(embeds_type)

    df_embs = daft.from_pydict({"embeddings": embeds_series,
    "img_path_nouri":  df.select("img_path_nouri").to_arrow()["img_path_nouri"]}
    )

    df = df.join(df_embs, on="img_path_nouri")

    return df.exclude("img_path_nouri")

In [ ]:
#| hide

from bedmap.prepare_images import df_images_from_pattern
from bedmap.config import Cfg

cfg = Cfg()
TEST_DIR = "../tests/test-data/smithsonian_butterflies_10/jpgs"

df = df_images_from_pattern(TEST_DIR)
df = create_embeddings_col(df, model_name=cfg.model_name, batch_size=BATCH_SIZE)

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/pydantic_settings/main.py:425: UserWarning: Config key `pyproject_toml_table_header` is set in model_config but will be ignored because no PyprojectTomlConfigSettingsSource source is configured. To use this config key, add a PyprojectTomlConfigSettingsSource source to the settings sources via the settings_customise_sources hook.
  self._settings_warn_unused_config_keys(sources, self.model_config)


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Project: 00:00 

Device set to use cpu
12it [00:04,  2.73it/s]                      


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Project: 00:00 

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()